In [6]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm

from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Process the Mueller Matrices of the folders

In [7]:
# set the parameters to run the script
data_folder_path = r'C:\Users\romai\Desktop\Freezing_dev'
# data_folder_path = r'C:\Users\romai\Documents\illustrators\pathology_protocol\imgs\effect_freezing'
directories = [data_folder_path]
calib_directory = r'C:\Users\romai\Desktop\calib'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

Traceback (most recent call last):
  File "C:\Users\romai\Documents\processingMM\src\processingmm\batch_processing.py", line 70, in find_folder_name
    assert len(re.findall("[\d]{4}-[\d]{2}-[\d]{2}", root)) == 1
AssertionError
0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [8]:
time_base = 'T0_'
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path, time_base)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:18<00:00,  1.56s/it]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [9]:
import time

# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in tqdm(base_dirs):
    propagation_lists = {}
    
    for tissue_type in tissue_types:
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_lists[tissue_type] = selection_of_ROIs.square_selection(path_folder_temp, path_folder, WM, 
                                                                            tissue_type, path_align_folder)
    # 3. actually do the alignment
    propagation_lists = automatic_ROI_propagation.generate_combined_mask(propagation_lists, path_align_folder)
    automatic_ROI_propagation.do_alignment(path_align_folder)
    output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
    
    # 4. and collect the data after propagation
    for tissue_type in tissue_types:
        WM = tissue_type == 'WM'
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_lists[tissue_type], 
                                                            output_folders, data_folder_path, time_base)

 33%|███████████████████████████▋                                                       | 2/6 [03:36<07:00, 105.05s/it]

Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...
Folder already exists, trying again...


 50%|██████████████████████████████████████████                                          | 3/6 [04:48<04:30, 90.01s/it]

Folder already exists, trying again...
Folder already exists, trying again...


 67%|████████████████████████████████████████████████████████                            | 4/6 [06:04<02:48, 84.45s/it]

Folder already exists, trying again...
Folder already exists, trying again...


 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [07:23<01:22, 82.74s/it]

Folder already exists, trying again...


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [08:44<00:00, 87.36s/it]


## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [10]:
from freezingeffect import parameter_comparaison

In [11]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_, max_nb_of_squares = parameter_comparaison.get_parameters()
max_nb_of_squares = max_nb_of_squares * len(base_dirs)

# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders[0:5] + data_folders[-5:]

['C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_10',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_11',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_12',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_13',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_5',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_6',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S6_1\\polarimetry\\550nm\\50x50_images\\WM_7',
 'C:\\Users\\romai\\Desktop\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR

#### 4.2. Get the data for the measurement

In [12]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['WM'][list(data_fixation['WM'].keys())[4]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,90.438714,48.826117,88.988764,90.438714,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.951006,0.008035,0.946835,0.950960,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.033839,0.008290,0.030612,0.033405,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,13.718075,7.535706,9.113924,12.642457,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,138.557602,40.778178,139.550562,138.557602,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.954166,0.008465,0.946835,0.954474,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.035389,0.008608,0.030612,0.034738,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,17.549436,11.243671,13.670886,14.954019,None


In [13]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [14]:
data_fixation['WM'][list(data_fixation['WM'].keys())[0]]

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,143.981840,28.615985,143.595506,143.981840,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.940675,0.009160,0.932911,0.941054,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.030851,0.007455,0.020408,0.030462,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,33.000574,15.161536,28.481013,31.025037,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,144.829720,45.811760,137.528090,144.829720,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.943003,0.009989,0.932911,0.942482,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.039175,0.008820,0.030612,0.038309,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,16.255875,11.071572,13.670886,14.049432,None


In [15]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [16]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,31.025037,34.916452,11.884928,10.373068,12.642457,14.086276,19.361063,17.647239,15.161853,14.654944,...,18.756232,13.805369,23.502276,23.46591,12.628847,16.223314,13.939374,21.041658,14.430524,12.715518
1,14.049432,20.098765,12.752843,16.009744,14.954019,15.334452,NaN,13.268533,13.346692,17.802549,...,13.755702,19.153609,10.486526,13.96708,15.946302,12.954615,13.725447,14.874639,13.564546,12.682489


In [17]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times), azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, len(times))
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000
1,0.452842,0.575624,1.073026,1.543395,1.182841,1.088609,NaN,0.751876,0.880281,1.214781,...,0.733394,1.387403,0.446192,0.595207,1.262689,0.798518,0.984653,0.706914,0.93999,0.997402


#### 4.3. Statistical analysis

In [19]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [20]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z       p_value   n_1   n_2  before/GM  \
parameter      data_type time                                                
azimuth        GM        T0    2964.5  1.000000e+00  77.0  77.0  31.910607   
                         T1    5929.0  2.352306e-30  77.0  77.0  31.910607   
               WM        T0    3280.5  1.000000e+00  81.0  81.0  28.344631   
                         T1    6561.0  7.432443e-32  81.0  81.0  28.344631   
depolarization GM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    3600.0  6.382362e-01  90.0  77.0   1.000000   
               WM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    2430.0  4.800365e-05  90.0  81.0   1.000000   
retardance     GM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    4680.0  2.181267e-05  90.0  77.0   1.000000   
               WM        T0    4050.0  1.000000e+00  90.0  90.0   1.000000   
                         T1    4230.0  5.041549e-02  90.0  81.0   1.000000   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0        27.881361  31.910607     27.881361  
                         T1        27.881361   0.000000      0.000000  
               WM        T0        22.650352  28.344631     22.650352  
                         T1        22.650352   0.000000      0.000000  
depolarization GM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   1.005666      0.042217  
               WM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   1.002205      0.011499  
retardance     GM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   0.862620      0.432661  
               WM        T0         0.000000   1.000000      0.000000  
                         T1         0.000000   0.978276      0.312535

In [21]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [22]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_nb_of_squares)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_nb_of_squares)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_nb_of_squares)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_nb_of_squares)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_nb_of_squares)

retardance_cv

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,1.000000,1.000000,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.467031,1.399601,1.1351,0.578242,0.966382,1.198217,0.358436,0.303742,0.552756,0.973557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))